In [1]:
import inspect
from copy import deepcopy, copy
from inspect import Parameter, Signature
from types import FunctionType
from typing import *
from functools import lru_cache
from collections.abc import Mapping

from mousse import Dataclass, asdict, asclass
from mousse.types import get_fields_info, Accessor, Field, parse, parser

In [7]:
class ReadOnlyFieldException(Exception):
    def __init__(self, key: str):
        super().__init__(f"Field `{key}` is readonly")
        
        
class ConfigMetadata(Dataclass):
    readonly: bool = False
        
        
class Config(Dataclass, dynamic=True, accessor=Accessor):
    def __setattr__(self, key: str, val: Any):
        fields = get_fields_info(self.__class__, self)        

        if key in fields:
            metadata = _get_metadata(self)
            if metadata.readonly:
                raise ReadOnlyFieldException(key)
            metadata.readonly = True
            super()

        val = parse(Config, val)
        dtype = type(val)

        field = Field()
        field.annotation = dtype
        field.private = key.startswith("_")

        accessor = Accessor(key, field=field)
#         accessor.__set__(self, val)
        super().__setattr__(key, accessor)
        super().__setattr__(key, val)
        
        fields[key] = field


        
@parser(Config)
def parse_config(G: Type[Config], config: Union[Mapping, list, tuple, set]):
    if isinstance(config, (list, tuple, set)):
        return tuple(parse_config(G, elem) for elem in config)
    
    if isinstance(config, Mapping):
        data = {}
        for key, val in config.items():
            val = parse_config(Config, val)
            data[key] = val
        
        return Config(**data)
    
    return config


@lru_cache(maxsize=None)
def _get_metadata(obj: Any) -> ConfigMetadata:
    return ConfigMetadata()


def update(config: Config, **kwargs):
    for key, val in kwargs.items():
        setattr(config, key, val)



In [8]:
config = Config()
update(config, name="datnh21", items=[[{"aa": "banana"}]])

In [9]:
config

Config(name="datnh21", items=((Config(aa=<mousse.types.accessor.Accessor object at 0x10e5c4100>),),))

In [24]:
config

Config(name="datnh21", items=((Config(aa="banana"),),), yolo=1)

In [65]:
config.name = 2

ReadOnlyFieldException: Field name is readonly

In [7]:
TT

typing.Tuple[typing.Union[__main__.Config, typing.Any], ...]

In [11]:
type(Mapping)

abc.ABCMeta

In [11]:
b

Bar(foo_lst=[Foo(age=10, name="thu")], foo_dct={'a': Foo(age=11, name="ad"), 'b': Foo(age=12, name="fd")}, foo_nst=[{'a': Foo(age=11, name="ad"), 'b': Foo(age=12, name="fd")}], foo=Foo(age=14, name="dat"))

In [12]:
asdict(b)

{'foo_lst': [{'age': 10, 'name': 'thu'}],
 'foo_dct': {'a': {'age': 11, 'name': 'ad'}, 'b': {'age': 12, 'name': 'fd'}},
 'foo_nst': [{'a': {'age': 11, 'name': 'ad'}, 'b': {'age': 12, 'name': 'fd'}}],
 'foo': {'age': 14, 'name': 'dat'}}